In [ ]:
import sys
sys.path.append("../")  

In [25]:
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)
import torch.optim as optim
from torch import nn
from torch import Tensor
import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE

device(type='cuda', index=1)

In [ ]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 6

h_hyper_prmts["convlayers11"] = {"in_channels": 4096, "out_channels": 1024, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers14"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers1"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}
# h_hyper_prmts["convlayers12"] = {"in_channels": 512, "out_channels": 128, "use_bn": False, "drop_rate": 0.1}


# h_hyper_prmts["convlayers122"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.2}
# h_hyper_prmts["convlayers123"] = {"in_channels": 256, "out_channels": 128, "use_bn": False, "drop_rate": 0.2}


l_hyper_prmts["linerlayer1"] = {"in_channels":512, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12334"] = {"in_channels":512, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12"] = {"in_channels":512, "out_channels":256, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer123"] = {"in_channels":256, "out_channels":128, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer121"] = {"in_channels":128, "out_channels":64, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer31"] = {"in_channels":64, "out_channels":6, "use_bn":False, "drop_rate":0.05}



hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [ ]:
def loss_bs_matrix(outs, hg, device,weight):
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2
    return loss, loss_1, loss_2

In [ ]:
class Trainer(nn.Module):

    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

In [29]:
import dhg 
from dhg.data import Cooking200
data = Cooking200()
e_list = data["edge_list"]
num_v = data["num_vertices"]
G = dhg.Hypergraph(data["num_vertices"],data["edge_list"])
num_v, data["num_edges"]

(7403, 2755)

In [30]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=4096, out_features=1024, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=1024, out_features=512, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=512, out_features=512, bias=True)
      )
    )
  )
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.05, inplace=False)
  (4): Linear(in_features=512, out_features=512, bias=True)
  (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.05, inplace=False)
  (8): Linear(in_features=512, out_features=512, bias=T

In [31]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)

hgnn_trainer.weight = 400

In [36]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=4e-4, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(9000):
    if hgnn_trainer.weight > 5:
        hgnn_trainer.weight = hgnn_trainer.weight - 0.088
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"                , weight: {hgnn_trainer.weight}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: -1877.3966796875
                , loss1: -126.3563232421875
                , loss2: 6.828763580322265
                , weight: 14.911999999976425
in 10 epoch, average loss: -16890.553125
                , loss1: -1257.7578125
                , loss2: 1255.85693359375
                , weight: 14.031999999976433
in 20 epoch, average loss: -16781.8796875
                , loss1: -1260.38046875
                , loss2: 293.6218505859375
                , weight: 13.151999999976441
in 30 epoch, average loss: -15898.3515625
                , loss1: -1262.90361328125
                , loss2: 99.93917846679688
                , weight: 12.27199999997645
in 40 epoch, average loss: -14874.7734375
                , loss1: -1264.7095703125
                , loss2: 33.559774780273436
                , weight: 11.391999999976457
in 50 epoch, average loss: -13778.9359375
                , loss1: -1265.3216796875
                , loss2: 23.183242797851562
               

KeyboardInterrupt: 

In [37]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

1511

In [38]:
num_nodes:Tensor = outs_straight.sum(dim=0)
print(num_nodes.item())
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()

tensor([1233., 1234., 1234., 1234., 1234., 1234.], device='cuda:1',
       grad_fn=<SumBackward1>)


0.00013508037282182898